In [ ]:
# Works best with jupyter-notebook

In [ ]:
%matplotlib notebook 
#%matplotlib widget 
# https://ipython.readthedocs.io/en/stable/interactive/magics.html
import math
import numpy as np

from spatialmath import *
from spatialmath.base import *

import roboticstoolbox as rtb
from roboticstoolbox import *

import matplotlib.pyplot as plt
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# Lecture 6.7 Numeric Inverse Kinematics

In this notebook, we will begin to explore the numeric inverse kinematics for different robots.

---

Under the DHRobot class, we have a number of inverse kinemtic methods that we will study:
https://github.com/petercorke/robotics-toolbox-python/wiki/Kinematics
https://petercorke.github.io/robotics-toolbox-python/arm_dh.html

**Numeric method:**
- sol = ikine_LM(T, q0=None): Numerical inverse kinematics by Levenberg-Marquadt optimization (Robot superclass)

- sol = ikine_min(T, q0=None, qlim=True) which uses scipy.minimize with user cost function and stiffness. 

--- 

The elements of the tuple sol include at least:


Element   | Type 	      | Description 
:---|:---|:---
q 	      | ndarray (n)   | Joint coordinates for the solution, or None 
:---|:---|:---
success   | bool 	      | True if a solution found 
:---|:---|:---
reason 	  | str 	      | reason for failure 
:---|:---|:---
iterations| int 	      | number of iterations 
:---|:---|:---
residual  | float 	      | final value of cost function 

---

Note:

**numerical IK solvers** minimise a *scalar measure of error* between a starting and the desired end-effector pose. The measure is the squared-norm of a 6-vector comprising:

- translational error (a 3-vector)
- the orientation error as an Euler vector (angle/axis form encoded as a 3-vector

The starting joint angle configuration can be set to your current configuration or an entirely different guess. This can be passed into the q0 variable.

Numerical optimization is slower than analytical counterparts. For this toolbox, SciPy based mimimizers are used. They use a scalar cost measure and must *numerically compute a Jacobian* to find a solution. 


Let's start by creating a model for the Puma560 robot.

In [ ]:
puma = rtb.models.DH.Puma560()
print(puma)

Plot with joint angles qn.

In [ ]:
puma.plot(puma.qn);

Let us use the same example from before to designate a desired end-effector pose that we would like the puma robot to reach. 

In [ ]:
T = SE3(0.6, 0.1, 0.5)*SE3().RPY([0, 0, 0], unit='deg')

print(type(T))
print(T)

Plot the desired pose:

In [ ]:
trplot(T.A, dims=[-2,2], frame='EE', rviz=True); # Extracts numpy from T

This time however, we will use the numerical methods. Recall that our previous solution was equal to:
    - [3.057, 2.644,  0.03708, 0, -2.681, -3.057]

In [ ]:
sol = puma.ikine_LM(T)
print(sol)

Let's plot this:

In [ ]:
puma.plot(sol.q);

The above solution is quite different from what we got in the analytical solution. It is an optimization optimization that may reach a configuration that is quite different than that of the analytical solution.  

To learn more lower-level details of the numerical algorithm check out the two additional videos on numerical IKin's I placed in our [youtube playlist](https://www.youtube.com/watch?v=VhUA0jf7tI8&list=PLts2VQxwm4sy6VIJVtLBWL5oJMoTqGXh5&index=18). 

Test starting from different locations?

In [ ]:
sol = puma.ikine_LM(T, q0=puma.qn)
print(sol)

The residual is lower but the error is still substantial... plot and visualize.

In [ ]:
puma.plot(sol.q);

Furthermore, we are not able to choose a desired robot configuration, such as elbow-up...

We can do better when we have a goal configuration that is closer to the starting point.

As we did in notebook 6.6, we will create a sequences of desired joint angle configurations after using ctraj.\
Assign a final pose configuration:

In [ ]:
TF = SE3(0.6, 0.1, 1.0)*SE3().RPY([0, 0, 0], unit='deg')

Interpolate in task space:

In [ ]:
des_T = ctraj(T,TF,100)
print(len(des_T))

Compute ikine solns:

In [ ]:
des_q = puma.ikine_LM(des_T)
print(des_q[9])

Notice the lower error residual at the end... Why is this?

Get ready to plot. You will notice that the end-effector reaches the desired pose quite well.

In [ ]:
q = np.array([ des_q[i].q for i in range(0,len(des_q))])
print(type(q))

In [ ]:
puma.plot(q);

### Student Exercise
Study the solutions for a spherical wrist. For example... test different configurations where you vary:

- the roll angle from -90 to 90
- the pictch angle from -90 to 90
- the yaw angle from -90 to 90

i.e. roll = a=np.linspace(-90,90,181)
````    
T = SE3(2.0, 0.1, 0.5)*SE3().RPY([roll, 0, 0], unit='deg')\
````